In [14]:
!pip install qdrant-client
!pip install sentence-transformers
!pip install openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [15]:
import pandas as pd 

from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
from openai import OpenAI

In [16]:
df = pd.read_csv('./top_rated_wines.csv')
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)

700

In [17]:
# 1. Load a pretrained Sentence Transformer model
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings


qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance


In [18]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/var/folders/3y/5lptrc3107l6vgh7khxqmsyc0000gn/T/ipykernel_22182/3075695827.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [19]:
# vectorize!
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [20]:
user_prompt = "Suggest me wines from Australia"


In [21]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=10
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Greenock Creek Alices Shiraz 2002', 'region': 'Barossa Valley, Barossa, South Australia, Australia', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Rich and fleshy, with pretty coffee, plum, wild berry and spice notes that are smooth and polished, long and flavorful.  An extremely limited release wine from one of Australia\'s "Cult" wineries.'} score: 0.6388254823915379
{'name': "Greenock Creek Alice's Shiraz 2003", 'region': 'Barossa Valley, Barossa, South Australia, Australia', 'variety': 'Red Wine', 'rating': 97.0, 'notes': 'Rich and fleshy, with pretty coffee, plum, wild berry and spice notes that are smooth and polished, long and flavorful.  An extremely limited release wine from one of Australia\'s "Cult" wineries.'} score: 0.6388254823915379
{'name': 'Domaine du Coulet Cornas Billes Noires 2009', 'region': 'Cornas, Rhone, France', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'This is a concentrated and austere wine during its youth.  Minerality is expressed throug

In [22]:

# define a variable to hold the search results
search_results = [hit.payload for hit in hits]
search_results

[{'name': 'Greenock Creek Alices Shiraz 2002',
  'region': 'Barossa Valley, Barossa, South Australia, Australia',
  'variety': 'Red Wine',
  'rating': 96.0,
  'notes': 'Rich and fleshy, with pretty coffee, plum, wild berry and spice notes that are smooth and polished, long and flavorful.  An extremely limited release wine from one of Australia\'s "Cult" wineries.'},
 {'name': "Greenock Creek Alice's Shiraz 2003",
  'region': 'Barossa Valley, Barossa, South Australia, Australia',
  'variety': 'Red Wine',
  'rating': 97.0,
  'notes': 'Rich and fleshy, with pretty coffee, plum, wild berry and spice notes that are smooth and polished, long and flavorful.  An extremely limited release wine from one of Australia\'s "Cult" wineries.'},
 {'name': 'Domaine du Coulet Cornas Billes Noires 2009',
  'region': 'Cornas, Rhone, France',
  'variety': 'Red Wine',
  'rating': 96.0,
  'notes': 'This is a concentrated and austere wine during its youth.  Minerality is expressed throughout the palate and on 

In [23]:
# If using GPT API; Be sure to set the env variable and then run this line

# client = OpenAI()

In [24]:
# Run LLM locally with Llamafile with command: 
    # ./llava-v1.5-7b-q4.llamafile

# https://github.com/Mozilla-Ocho/llamafile?tab=readme-ov-file
# https://www.notion.so/Week-3-cc979d673e784975aa37afa507cfdaf9

In [25]:

client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)

completion = client.chat.completions.create(
    model="llama.cpp",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Help me pick a great wine from Australia and describe it's taste"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)



ChatCompletionMessage(content="Sure, I'd be happy to help you pick a great wine from Australia. One of the most highly regarded wineries in Australia is Greenock Creek, known for producing some of the best Shiraz wines in the country. Their 2002 and 2003 vintages are particularly sought after and highly rated by wine critics. These wines are rich and fleshy with notes of coffee, plum, wild berry and spice, and are known for their smooth and polished finish.\n\nAnother option to consider is Domaine du Coulet Cornas Billes Noires 2009. This wine is known for its concentrated and austere profile, with mineral notes throughout the palate and on the finish.\n\nIf you're looking for a more expensive option, the Domaine de la Romanee-Conti Grands Echezeaux Grand Cru 2005 is a highly regarded wine from Burgundy, France. It is known for its 96-97 point rating from Wine Advocate.\n\nFor a more unique option, Abreu Vineyards Howell Mountain 2009 is a great choice. This wine is known for its full-